In [7]:
from ezc3d import c3d
from pyomeca import Markers, Analogs
import config as cfg
import importlib

def normalize_emg(emg):
    emg_p = (
    emg.meca.band_pass(order=2, cutoff=[10, 425])
    .meca.center()
    .meca.abs()
    .meca.low_pass(order=4, cutoff=5, freq=emg.rate)
    .meca.normalize(ref=None, scale=1)
    )
    return emg_p

def name_body(data_path, marker_list):
    body = data_path.split('-')[3]+":"
    for i in range(len(marker_list)):
        marker_list[i] = body+marker_list[i]
    return marker_list

def data_markers(data_path, marker_list):
    for i in range(len(marker_list)):
        if i == 0:
            data_markers = Markers.from_c3d(data_path, usecols=[marker_list[0]])
            data_markers = data_markers.meca.to_wide_dataframe()
        else:
            tmp_markers = Markers.from_c3d(data_path, usecols=[marker_list[i]])
            tmp_markers = tmp_markers.meca.to_wide_dataframe()
            data_markers = data_markers.join(tmp_markers)      
    cols = [c for c in data_markers.columns if c.lower()[:4] != 'ones']
    markers_dataframe = data_markers[cols]
    return markers_dataframe

def dataframe(data_path):
    mark_channels = name_body(data_path,cfg.marker_channels[:13])
    plat_channels = name_body(data_path,cfg.platform_channels)
    ang_channels = name_body(data_path,cfg.angles_channels)
    forc_channels = name_body(data_path,cfg.forces_channels)
    mome_channels = name_body(data_path,cfg.moments_channels)
    powe_channels = name_body(data_path,cfg.powers_channels)
    
    full_data = data_markers(data_path,mark_channels)
    full_data = full_data.join(data_markers(data_path,cfg.marker_channels[13:]))
    emg1 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[:9])
    emg2 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[9:])
    norm_emg1 = normalize_emg(emg1)
    norm_emg2 = normalize_emg(emg2)
    norm_emg1 = norm_emg1.meca.to_wide_dataframe()
    norm_emg2 = norm_emg2.meca.to_wide_dataframe()
    
    full_data = full_data.join(norm_emg1)
    full_data = full_data.join(norm_emg2)
    
    full_data = full_data.join(data_markers(data_path,plat_channels))
    full_data = full_data.join(data_markers(data_path,ang_channels))
    full_data = full_data.join(data_markers(data_path,forc_channels))    
    full_data = full_data.join(data_markers(data_path,mome_channels))
    
    data_powers_tmp = data_markers(data_path,powe_channels)
    cols = [c for c in data_powers_tmp.columns if c.lower()[:1] == 'z']
    data_powers = data_powers_tmp[cols]
    
    full_data = full_data.join(data_powers)
    
    return full_data

In [10]:
importlib.reload(cfg)
data_path = r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K5.c3d"
n = dataframe(data_path)

In [11]:
n

,x_B0444:LSHO,y_B0444:LSHO,z_B0444:LSHO,x_B0444:LELB,y_B0444:LELB,z_B0444:LELB,x_B0444:LWRA,y_B0444:LWRA,z_B0444:LWRA,x_B0444:RSHO,...,z_B0444:LAnklePower,z_B0444:RHipPower,z_B0444:RKneePower,z_B0444:RAnklePower,z_B0444:LShoulderPower,z_B0444:LElbowPower,z_B0444:LWristPower,z_B0444:RShoulderPower,z_B0444:RElbowPower,z_B0444:RWristPower
time,,,,,,,,,,,,,,,,,,,,,
0.000,-430.316956,-185.981323,1455.084595,-420.517090,-10.783237,1158.911133,-240.802902,-64.469841,1310.695801,-166.688095,...,-0.001600,0.005166,0.129711,0.011504,-0.000042,-0.001521,0.000055,-0.004303,-0.000009,0.000030
0.005,-430.261749,-185.387650,1455.095703,-420.612457,-10.227446,1158.878418,-240.738846,-63.851131,1310.471802,-166.672714,...,-0.001377,0.003972,0.127073,0.004990,-0.000174,-0.001638,0.000052,-0.004934,-0.000011,0.000032
0.010,-430.206818,-184.774658,1455.070923,-420.693878,-9.681876,1158.789185,-240.666245,-63.212677,1310.208618,-166.652176,...,-0.001157,0.002609,0.124446,-0.001744,-0.000295,-0.001764,0.000045,-0.005618,-0.000013,0.000036
0.015,-430.153015,-184.142181,1455.012207,-420.761597,-9.148438,1158.644409,-240.585571,-62.553543,1309.909668,-166.627335,...,-0.000940,0.000909,0.119840,-0.007804,-0.000411,-0.001901,0.000035,-0.006354,-0.000015,0.000040
0.020,-430.101196,-183.490036,1454.921021,-420.815796,-8.628656,1158.445679,-240.497131,-61.872982,1309.578003,-166.599091,...,-0.000724,-0.000942,0.116576,-0.013543,-0.000515,-0.002047,0.000023,-0.007147,-0.000017,0.000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.510,-468.259247,-161.368652,1438.250732,-463.628418,2.122261,1135.995117,-274.336212,-32.887882,1283.627441,-196.013306,...,0.022741,-0.033089,0.309469,-0.111663,-0.010878,-0.003979,0.000028,-0.027947,0.000383,0.000027
3.515,-469.228394,-159.165588,1437.672119,-465.257568,4.205086,1135.295166,-275.579987,-30.268538,1282.551514,-196.799622,...,0.023112,-0.029463,0.326531,-0.113147,-0.010335,-0.004045,0.000024,-0.029487,0.000409,0.000008
3.520,-470.233215,-156.964081,1437.068115,-466.972473,6.302920,1134.579712,-276.894745,-27.593353,1281.446045,-197.586136,...,0.023619,-0.018552,0.332486,-0.113016,-0.009696,-0.004140,0.000019,-0.031072,0.000434,-0.000013
